<img src="https://cybersecurity-excellence-awards.com/wp-content/uploads/2017/06/366812.png">


<h1><center>Darwin Unsupervised Model Building </center></h1>


Prior to getting started, there are a few things you want to do:
1. Enter your username and password to ensure that you're able to log in successfully
2. Set the path to your dataset. If left unfilled, you will be testing an example dataset on the server. 
3. Set the dataset path for feature importance
  - For global feature importance, the dataset path remains the same as your original dataset
  - For individual row's feature importance, you need to specify a path to a dataset that contains no more than 500       rows.

Once you're up and running, here are a few things to be mindful of:
1. For every run, check the job status (i.e. requested, failed, running, completed) and wait for job to complete before proceeding. 
2. If you're not satisfied with your model and think that Darwin can benefit from extra training, use the resume function.

## Import libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os

from amb_sdk.sdk import DarwinSdk

## Setup
In the cell below, you need to set up the following variables:
 - path: the designated path to your dataset, default to be an example dataset "pulsar.csv" which you should be able to find in the Darwin SDK /sets folder
 - user, password: your credentials to log in the next time you use Darwin SDK. These should be inside of your trial email.

In [ ]:
# set local data path to files
path = '../../sets/'

# View data snippet
df = pd.read_csv(os.path.join(path, 'pulsars.csv'))
df.head()

Log in with the username and password that were attached to your trial email

In [ ]:
ds = DarwinSdk()
ds.set_url('https://amb-trial-api.sparkcognition.com/v1/')
status, msg = ds.auth_login_user('username','password')
if not status:
    print(msg)
else:
    print("You are now logged in!")

## Upload Data
After setting up the dataset path, the next step is to upload the dataset from your local device to the server. In the cell below, you need to specify the dataset_name.

In [ ]:
# Upload dataset
ds.delete_all_models()
ds.delete_all_datasets()
dataset_name = 'pulsars'
status, dataset = ds.upload_dataset(os.path.join(path, 'pulsars.csv'), dataset_name)
if not status:
    print(dataset)
else:
    print("Data uploaded successfully")

## Create and Train Model 

To build unsupervised models, which cluster data and perform anomaly detection, Darwin goes through the following steps:
1. Determines an approximate number of clusters to start with using a single pass with a hierarchical method
2. Iterates on subsets of the data using a Spectral-Net algorithm to determine the ideal number of clusters
3. Proceeds to cluster the data using a Spectral-Net approach

In the cell below, specify the parameters used to create the model:
- model: the name of your model
- max_epochs: the number of epochs to train the model, one epoch indicates one scan of the entire dataset
- n_clusters: the number of clusters, either an integer or 'auto', if left with 'auto', the unsupervised algorithm will compute a number for you

In [ ]:
# Build model
model = "your_model_name"
max_epochs = 20
n_clusters = 2
status, job_id = ds.create_model(dataset_names=dataset_name,
                                 model_name=model,
                                 max_epochs=max_epochs,
                                 n_clusters=n_clusters)
sleep(1)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

In [ ]:
# look up job status
ds.lookup_job_status_name(job_id['job_name'])

In [ ]:
# look up the model
ds.lookup_model_name(job_id['model_name'])

## Extra Training (Optional)
Run the following cell for extra training, no need to specify parameters

In [ ]:
# Train some more
extra_epochs = 10
status, job_id = ds.resume_training_model(dataset_names=dataset_name,
                                          model_name=model,
                                          max_epochs=extra_epochs,
                                          n_clusters=n_clusters)
sleep(1)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

## Predict
Run the following cell for prediction

In [ ]:
# Test model
status, artifact = ds.run_model(dataset_name, 
                                model, 
                                supervised=False)
sleep(1)
ds.wait_for_job(artifact['job_name'])

In [ ]:
# Get predictions
status, pred_file = ds.download_artifact(artifact['artifact_name'])

In [ ]:
# View prediction
pred_file.head()

## Analyze Data

In [ ]:
status, analyze_id = ds.analyze_data(dataset_name, 
                                     job_name = 'Darwin_analyze_data_job', 
                                     artifact_name = 'Darwin_analyze_data_artifact')
sleep(1)
if status:
    ds.wait_for_job('Darwin_analyze_data_job')
else:
    print(analyze_id)

In [ ]:
ds.lookup_job_status_name('Darwin_analyze_data_job')

In [ ]:
status, analyze_results = ds.download_artifact('Darwin_analyze_data_artifact')
analyze_results

## Analyze Model
Analyze model provides feature importance ranked by the model. It indicates a general view of which features pose a bigger impact on the model

In [ ]:
status, analyze_id = ds.analyze_model(job_id['model_name'], 
                                      job_name='Darwin_analyze_model_job', 
                                      artifact_name='Darwin_analyze_model_artifact')
sleep(1)
if status:
    ds.wait_for_job('Darwin_analyze_model_job')
else:
    print(analyze_id)

In [ ]:
ds.lookup_job_status_name('Darwin_analyze_model_job')

Downloade and print the top 10 features

In [ ]:
status, feature_importance = ds.download_artifact('Darwin_analyze_model_artifact')
feature_importance

## Analyze Prediction
Different from Analyze Model, the Analyze Prediction provides a way to analyze feature importance for each data point. The output estimates how each feature added or subtracted from a known base-value to result in the overall prediction that was made. <br>
**You need to set the path to a dataset which contains all the samples you want to analyze (max rows = 500)**

In [ ]:
# Upload the data that you are interested in feature importance (max: 500 rows)
dataset_name = 'pulsars_predict'
path = '../../sets/'
status, response = ds.upload_dataset(os.path.join(path, 'pulsars_predict.csv'), dataset_name)
print(status)
print(response)
if status:
    dataset_by_row=response['dataset_name']
else:
    print("Upload data failed!")

In [ ]:
status, analyze_id = ds.analyze_predictions(job_id['model_name'], 
                                            'pulsars_predict',
                                            job_name='Analyze_prediction_job', 
                                            artifact_name='Analyze_prediction_artifact')
sleep(1)
if status:
    ds.wait_for_job('Analyze_prediction_job')
else:
    print(analyze_id)

In [ ]:
ds.lookup_job_status_name('Analyze_prediction_job')

Download and print the top 10 features

In [ ]:
status, feature_importance = ds.download_artifact('Analyze_prediction_artifact')
feature_importance.head()